In [128]:
#!pip install biopython
#        OR
#!pip3 install biopython

In [2]:
import os
from Bio import SeqIO
from Bio.Align import MultipleSeqAlignment

# Function to parse PDB file and extract secondary structure information
def parse_pdb(pdb_file):
    helix_records = []
    sheet_records = []

    with open(pdb_file, 'r') as f:
        lines = f.readlines()

    for line in lines:
        if line.startswith("HELIX"):
            helix_records.append({
                'beg_label_seq_id': int(line[21:25].strip()),
                'end_label_seq_id': int(line[33:37].strip()),
                'label_asym_id': line[19].strip(),
            })
        elif line.startswith("SHEET"):
            sheet_records.append({
                'beg_label_seq_id': int(line[22:26].strip()),
                'end_label_seq_id': int(line[33:37].strip()),
                'label_asym_id': line[21].strip(),
            })
    return helix_records, sheet_records

def map_secondary_structure_to_alignment(alignment, secondary_structure_dict):
    def is_in_region(residue_index, regions):
        for region in regions:
            beg_seq_id = int(region['beg_label_seq_id'])
            end_seq_id = int(region['end_label_seq_id'])
            if beg_seq_id <= residue_index <= end_seq_id:
                return True
        return False
    
    def color_chain(annotated_seq, start_index, end_index, color):
        for i in range(start_index, end_index + 1):
            annotated_seq[i] = color
        return annotated_seq

    ss_annotations = []

    for record in alignment:
        chain_id = record.id.split(':')[-1]  # Extract chain ID from sequence ID
        annotated_seq = []
        if chain_id in secondary_structure_dict:
            helix_records, sheet_records = secondary_structure_dict[chain_id]
            residue_index = 0  # Track the position of residues that are not dashes
            for i, residue in enumerate(record.seq):
                if residue != '-':
                    residue_index += 1
                    if is_in_region(residue_index, helix_records):
                        annotated_seq.append("#f4b899")  # Helix
                    elif is_in_region(residue_index, sheet_records):
                        annotated_seq.append("#87cfea")  # Sheet
                    else:
                        annotated_seq.append("#ffffff")  # Not in Helix or Sheet
                else:
                    annotated_seq.append("-")  # Leave dash line uncolored
        else:
            # If chain_id is not in the dictionary, mark all as "-"
            annotated_seq = ["-"] * len(record.seq)
        
        ss_annotations.append(annotated_seq)
    
    # Iterate over ss_annotations to merge consecutive dashes between the same colored regions
    for i in range(len(ss_annotations)):
        j = 0
        while j < len(ss_annotations[i]):
            if ss_annotations[i][j] == '-':
                start_chain = j
                while j < len(ss_annotations[i]) and ss_annotations[i][j] == '-':
                    j += 1
                end_chain = j - 1
                
                # Determine the color before and after the sequence of dashes
                if start_chain > 0:
                    prev_color = ss_annotations[i][start_chain - 1]
                else:
                    prev_color = None
                
                if end_chain < len(ss_annotations[i]) - 1:
                    next_color = ss_annotations[i][end_chain + 1]
                else:
                    next_color = None
                
                # If the dashes are between the same colored regions, color them accordingly
                if prev_color == next_color and prev_color is not None:
                    ss_annotations[i] = color_chain(ss_annotations[i], start_chain, end_chain, prev_color)
            else:
                j += 1
    
    return ss_annotations

# Function to generate HTML representation of colored alignment (horizontal)
def color_alignment_html_rotated(alignment, ss_annotations, pdb_file_names, output_file):
    num_sequences = len(alignment)
    sequence_length = len(alignment[0])

    # Initialize HTML content
    html_content = "<html><body><pre>"

    # Determine the maximum length of PDB file names for alignment
    max_pdb_length = max(len(name) for name in pdb_file_names)

    # Loop through each sequence in the alignment
    for i in range(num_sequences):
        # Get PDB file name based on current sequence index
        pdb_file_name = pdb_file_names[i]

        # Calculate padding to align sequences by starting PDB file name
        padding = ' ' * (max_pdb_length - len(pdb_file_name))

        # Add PDB file name and padding to the left of the alignment
        html_content += f"<b>{pdb_file_name}</b>{padding}"

        for j in range(sequence_length):
            ss = ss_annotations[i][j]
            residue = alignment[i][j]

            # Use inline style to set background color behind the text
            html_content += (
                f'<span style="background-color:{ss}; padding: 2px; display: inline-block;">{residue}</span>'
            )

        html_content += "<br>"

    html_content += "</pre></body></html>"

    # Write HTML content to file
    with open(output_file, 'w') as f:
        f.write(html_content)

    print(f"Rotated colored alignment saved to {output_file}")

# Function to determine the format of the alignment file
def determine_format(file_path):
    _, ext = os.path.splitext(file_path)
    if ext.lower() == '.fas' or ext.lower() == '.fasta':
        return 'fasta'
    elif ext.lower() == '.aln':
        return 'clustal'
    else:
        raise ValueError(f"Unsupported alignment file format: {ext}")

Rotated colored alignment saved to aln-Test.html


### Replace the path with the path of the file you want to read

In [130]:

# Directory containing PDB files
pdb_files_dir = "Replace_With_PDB FOlders_Relative_Path"  # e.g., "PDB_files/"

# Read the multi-sequence alignment file could be .fas or .aln
alignment_file = "Replace_With_Alignment_Relative_Path"  # e.g., "alignment.fasta"

In [131]:
# Determine the format of the alignment file
alignment_format = determine_format(alignment_file)

# List all PDB files in the directory
pdb_files = [os.path.join(pdb_files_dir, f) for f in os.listdir(pdb_files_dir) if f.endswith('.pdb')]

# Initialize a dictionary to store secondary structure annotations
secondary_structure_dict = {}

# Loop through each PDB file and parse the secondary structure information
for pdb_file in pdb_files:
    helix_records, sheet_records = parse_pdb(pdb_file)
    # Use the PDB file name (without extension) as the key
    secondary_structure_dict[os.path.basename(pdb_file).split('.')[0]] = (helix_records, sheet_records)

# Read the alignment
alignment = SeqIO.parse(alignment_file, alignment_format)
alignment = MultipleSeqAlignment(alignment)

# Extract PDB file names from the alignment
pdb_file_names = [seq.id for seq in alignment]

# Verify that each PDB file name from the alignment has a corresponding entry in the secondary structure dictionary
missing_files = [name for name in pdb_file_names if name not in secondary_structure_dict]
if missing_files:
    print(f"Warning: The following PDB files are missing or could not be parsed: {missing_files}")

# Map secondary structure annotations to the alignment
ss_annotations = map_secondary_structure_to_alignment(alignment, secondary_structure_dict)

# Generate the HTML representation
output_file = "Output.html"
color_alignment_html_rotated(alignment, ss_annotations, pdb_file_names, output_file)

Rotated colored alignment saved to Output.html


### The file should be in the same directory as the notebook, called Output.html